In [1]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action='ignore')
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


날씨와 농산물 가격은 어떤 상관관계가 있는지 예측한다.  
기상정보: 기상자료개방포털(https://data.kma.go.kr/), 농산물 가격정보: 농산물유통정보(https://www.kamis.or.kr/)
***
기상정보: 평균기온(avgTemp), 최저기온(minTemp), 최고기온(maxTemp), 강수량(rainFall)  
농산물가격정보: 배추가격(avgPrice)

In [2]:
# price_data.csv 파일을 읽어서 판다스 데이터프레임으로 저장한다.
# csv 파일의 첫 줄에 컬럼 이름(헤더)이 없을 경우는 아래와 같이 읽어서 컬럼 이름을 붙여준다.
# price_data = pd.read_csv('./data/price_data.csv', header=None)
# columns = ['year', 'avgTemp', 'minTemp', 'maxTemp', 'rainFall', 'avgPrice']
# price_data.columns = columns
price_data = pd.read_csv('./data/price_data.csv')
# print(type(price_data))
price_data

,year,avgTemp,minTemp,maxTemp,rainFall,avgPrice
0,20100101,-4.9,-11.0,0.9,0.0,2123
1,20100102,-3.1,-5.5,5.5,0.8,2123
2,20100103,-2.9,-6.9,1.4,0.0,2123
3,20100104,-1.8,-5.1,2.2,5.9,2020
4,20100105,-5.2,-8.7,-1.8,0.7,2060
...,...,...,...,...,...,...
2917,20171227,-3.9,-8.0,0.7,0.0,2865
2918,20171228,-1.5,-6.9,3.7,0.0,2884
2919,20171229,2.9,-2.1,8.0,0.0,2901
2920,20171230,2.9,-1.6,7.1,0.6,2901


In [3]:
# 데이터프레임에 저장된 데이터를 텐서플로우에서 처리하기에 적합하도록 넘파이 배열 형태로 변환한다.
data = np.array(price_data, dtype=np.float32)
# print(type(data))
data

array([[ 2.0100100e+07, -4.9000001e+00, -1.1000000e+01,  8.9999998e-01,
         0.0000000e+00,  2.1230000e+03],
       [ 2.0100102e+07, -3.0999999e+00, -5.5000000e+00,  5.5000000e+00,
         8.0000001e-01,  2.1230000e+03],
       [ 2.0100104e+07, -2.9000001e+00, -6.9000001e+00,  1.4000000e+00,
         0.0000000e+00,  2.1230000e+03],
       ...,
       [ 2.0171228e+07,  2.9000001e+00, -2.0999999e+00,  8.0000000e+00,
         0.0000000e+00,  2.9010000e+03],
       [ 2.0171230e+07,  2.9000001e+00, -1.6000000e+00,  7.0999999e+00,
         6.0000002e-01,  2.9010000e+03],
       [ 2.0171232e+07,  2.0999999e+00, -2.0000000e+00,  5.8000002e+00,
         4.0000001e-01,  2.9010000e+03]], dtype=float32)

In [4]:
# 넘파이 배열에서 변화 요인 데이터(평균기온, 최저기온, 최고기온, 강수량)로 사용할 데이터를 뽑아낸다.
xData = data[:, 1:5]
# print(type(xData))
xData

array([[ -4.9, -11. ,   0.9,   0. ],
       [ -3.1,  -5.5,   5.5,   0.8],
       [ -2.9,  -6.9,   1.4,   0. ],
       ...,
       [  2.9,  -2.1,   8. ,   0. ],
       [  2.9,  -1.6,   7.1,   0.6],
       [  2.1,  -2. ,   5.8,   0.4]], dtype=float32)

In [5]:
# 넘파이 배열에서 결과(평균가격)로 사용할 데이터를 뽑아낸다.
# yData = data[:, 5] # 인덱싱으로 뽑아내면 1차원 데이터로 뽑아낸다.
# 변화 요인이 2차원이므로 결과도 2차원 형태로 뽑아내야 한다.
# yData = data[:, 5:6] # 슬라이싱으로 뽑아내면 2차원 데이터로 뽑아낸다.
yData = data[:, [5]]
# print(type(yData))
yData

array([[2123.],
       [2123.],
       [2123.],
       ...,
       [2901.],
       [2901.],
       [2901.]], dtype=float32)

In [6]:
# 뽑아낸 데이터를 텐서플로우에서 처리하기 위해서 placeholder를 만든다.
X = tf.placeholder(dtype=tf.float32, shape=[None, 4]) # 변화요인(평균기온, 최저기온, 최고기온, 강수량)
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) # 가격(실제값)

In [7]:
# 다변인 선형 회귀 모델의 기울기(가중치)와 y절편(바이어스)을 임의의 값으로 초기화한다.
a = tf.Variable(tf.random_uniform([4, 1]), dtype=tf.float32) # 기울기, 4행 1열의 난수로 발생시킨다.
b = tf.Variable(tf.random_uniform([1]), dtype=tf.float32) # y절편
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('가중치\n', sess.run(a), sep='')
print('y절편\n', sess.run(b), sep='')

가중치
[[0.7213738 ]
 [0.62180185]
 [0.5066409 ]
 [0.01786518]]
y절편
[0.8384086]


In [8]:
# 행렬의 곱셈 연산을 이용해서 다변인 선형 회귀 모델의 가설 식을 세운다. => 예측값을 계산하는 식
y = tf.matmul(X, a) + b # 예측값
# 오차함수를 만든다. => 예측값(y)과 실제값(Y)의 편차의 제곱에 대한 평균 => 평균 제곱법
loss = tf.reduce_mean(tf.square(y - Y))
# 경사하강법 알고리즘을 사용해서 오차함수의 결과를 최소로 하는 기울기와 y절편을 찾는다.
gradient_descent = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [9]:
# 학습시킨다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(100001):
    loss_, y_, _ = sess.run([loss, y, gradient_descent], feed_dict={X: xData, Y: yData})
    if epoch % 5000 == 0:
        # print('epoch: %6d, loss: %12.3f, price: %8.2f' % (epoch, loss_, y_[0]))
        print('epoch: {:6d}, loss: {:12.3f}, price: {:8.2f}'.format(epoch, loss_, y_[0][0]))

epoch:      0, loss: 12420579.000, price:    -7.05
epoch:   5000, loss:  1965985.375, price:  2649.74
epoch:  10000, loss:  1873647.750, price:  2720.10
epoch:  15000, loss:  1842602.625, price:  2771.75
epoch:  20000, loss:  1831589.250, price:  2805.98
epoch:  25000, loss:  1827613.375, price:  2827.65
epoch:  30000, loss:  1826169.125, price:  2841.07
epoch:  35000, loss:  1825644.125, price:  2849.28
epoch:  40000, loss:  1825453.250, price:  2854.27
epoch:  45000, loss:  1825383.750, price:  2857.29
epoch:  50000, loss:  1825358.125, price:  2859.12
epoch:  55000, loss:  1825349.000, price:  2860.22
epoch:  60000, loss:  1825345.875, price:  2860.89
epoch:  65000, loss:  1825344.875, price:  2861.29
epoch:  70000, loss:  1825344.250, price:  2861.52
epoch:  75000, loss:  1825344.000, price:  2861.69
epoch:  80000, loss:  1825343.625, price:  2861.73
epoch:  85000, loss:  1825344.000, price:  2861.74
epoch:  90000, loss:  1825344.000, price:  2861.74
epoch:  95000, loss:  1825344.0

In [10]:
# 테스트
data = [[5.2, -7.5, 10.1, 15.0]]
result = sess.run(y, feed_dict={X: data})
print('평균온도: {:4.1f}, 최저온도: {:4.1f}, 최고온도: {:4.1f}, 강수량: {:4.1f}일 때 '\
      '배추 가격은 {:7,.2f}원 입니다.'.format(data[0][0], data[0][1], data[0][2], data[0][3], result[0][0]))

평균온도:  5.2, 최저온도: -7.5, 최고온도: 10.1, 강수량: 15.0일 때 배추 가격은 2,802.26원 입니다.


In [11]:
# 학습 완료된 예측 모델을 디스크에 저장한다.
# Saver() 함수로 텐서플로우에서 학습된 모델을 디스크에 저장 또는 불러올 때 사용하는 객체를 생성한다.
saver = tf.train.Saver()
saver.save(sess, './model/saved.cpkt')
print('학습된 모델을 저장했습니다.')

학습된 모델을 저장했습니다.


<img src="./model.png" width="800" align="left"/>